# Exercises    
    For each of the following questions, formulate a null and alternative hypothesis (be as specific as you can be), then give an example of what a true positive, true negative, type I and type II errors would look like. Note that some of the questions are intentionally phrased in a vague way. It is your job to reword these as more precise questions that could be tested.

1. Has the network latency gone up since we switched internet service providers?
- null : latency performance is not related to internet providers 
- althypo : latency performance is related to internet providers 
- truepos : avg latency from one company is significantly higher than the other
- trueneg : avg latency from one company is not significantly higher than the other
- type1 : latency is better with one company statistically, but in reality it isnt
- type2 : company doesn't seem to matter with our tests, but in reality actually does
2.  Is the website redesign any good? (just as a comment, what does good even mean? For this I am assuming amt of users visiting site)
- null : website views are not related to the website change
- althypo : website views are related to website change
- truepos : website views are higher with the redesigned group compared to original
- falsepos : website views on original are higher than the redesigned
- type1 : views are higher on the new design, but in actuality it's due to just more people visiting, unrelated to the design
- type2 : our redesign doesn't change our engagement, but in reality it increases user engagement and views. 
3. Is our television ad driving more sales?
- null: advertising campaign does not influence sales
- althypo : advertising campaign influenced sales
- truepos : advertising campaing did create a larger amount of sales
- falsepos : advertising campaign did not increase sales
- type1 : sales are up, but not due to the ad. maybe its the weather
- type2 : sales aren't up, but that's due to people not needing our product. The ad is getting new customers, they just dont increase our sales enough. 

In [52]:
import numpy as np
import seaborn as sns
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
from pydataset import data

Ace Realty wants to determine whether the average time it takes to sell homes is different for its two offices. A sample of 40 sales from office #1 revealed a mean of 90 days and a standard deviation of 15 days. A sample of 50 sales from office #2 revealed a mean of 100 days and a standard deviation of 20 days. Use a .05 level of significance.



- Null: time to sell a home is not different between groups
- Alternate: Time to sell a home is different

In [3]:
stats.ttest_ind_from_stats(90,15,40,100,20,50, equal_var = False)

Ttest_indResult(statistic=-2.7091418459143854, pvalue=0.00811206270346016)

I used ttest from stats since I had some of the descriptive stats. Negative test statistic and p value < .05 means we can reject the null hypothesis that time of sale is not significantly different. The negative test lets us know that office 2 takes longer to sell homes. 

Load the mpg dataset and use it to answer the following questions:

Is there a difference in fuel-efficiency in cars from 2008 vs 1999?

In [60]:
#import data
df = data('mpg')
#rename columns
df = df.rename(columns = {'cty':'city', 'hwy':'highway', 'cyl':'cylinder', 'drv':'drive'})
# create new columns for average_mileage and mileage_difference. 
df['average_mileage'] = df[['highway','city']].mean(axis = 1)
df['harmonic_mileage'] = stats.hmean(df[['city', 'highway']], axis = 1)
df['mileage_difference'] = df['highway'].sub(df['city'], axis = 0)

In [61]:
#create new data frames that filter by date
old_cars = df[df['year'] == 1999]
new_cars = df[df['year'] == 2008]
stats.levene(old_cars["average_mileage"], new_cars["average_mileage"])

LeveneResult(statistic=0.033228136671080453, pvalue=0.855517569468803)

The levene test determined that the variances of both old and new cars are not significantly different. 

In [62]:
## using harmonic mean
stats.levene(old_cars["harmonic_mileage"], new_cars["harmonic_mileage"])

LeveneResult(statistic=0.031071867429515194, pvalue=0.8602339200830025)

In [18]:
stats.ttest_ind(old_cars["average_mileage"], new_cars["average_mileage"])

Ttest_indResult(statistic=0.21960177245940962, pvalue=0.8263744040323578)

The differences in average gas mileage between the two groups is not significantly different enough to reject the null hypothesis. 

In [63]:
## using harmonic mean
stats.ttest_ind(old_cars["harmonic_mileage"], new_cars["harmonic_mileage"])

Ttest_indResult(statistic=0.3011962975077886, pvalue=0.7635345888327115)

Are compact cars more fuel-efficient than the average car?

- Null: mpg of compact cars are not different than the population mean
- Alternative : mpg of compact cars are different than the population mean

In [64]:
# create mask to filter compact cars to compare to all the other classes of cars
compact_mask = df["class"] == "compact"
compact = df[compact_mask]
avg_car = df[compact_mask == False]

In [65]:
df["class"].unique()

array(['compact', 'midsize', 'suv', '2seater', 'minivan', 'pickup',
       'subcompact'], dtype=object)

In [67]:
stats.levene(compact["average_mileage"], avg_car["average_mileage"])

LeveneResult(statistic=11.107991264810888, pvalue=0.001000307619344077)

Levene's test indicates that the variance between both groups are significantly different. 

In [68]:
stats.levene(compact["harmonic_mileage"], avg_car["harmonic_mileage"])

LeveneResult(statistic=7.730287593946991, pvalue=0.005875923121564377)

In [53]:
stats.ttest_ind(compact["average_mileage"], avg_car["average_mileage"], equal_var = False)

Ttest_indResult(statistic=8.128810422808078, pvalue=1.6018060656123074e-12)

In [54]:
stats.ttest_1samp(compact["average_mileage"],df["average_mileage"].mean())

Ttest_1sampResult(statistic=7.896888573132535, pvalue=4.1985637943171336e-10)

At first, I did this as a independent t test due to misreading the question. I went back and ran the analysis again as a 1 sample. Both have p<0.05, so we can reject our null and confirm that compact cars have significantly better gas mileage

In [69]:
## harmonic mean
stats.ttest_1samp(compact["harmonic_mileage"],df["harmonic_mileage"].mean())

Ttest_1sampResult(statistic=7.512360093161354, pvalue=1.5617666348807727e-09)

Do manual cars get better gas mileage than automatic cars?

- Null: mpg of manual cars are not better than automatic cars
- Alternate: mpg of manual cars are better than automatic cars

In [71]:
auto = df[df['trans'].str.contains("auto")]
manual =  df[df['trans'].str.contains("manual")]

In [72]:
stats.levene(auto["average_mileage"], manual["average_mileage"])

LeveneResult(statistic=0.20075824847529639, pvalue=0.6545276355131857)

we can confirm that the difference in variances between the two groups is insignificant and thus are equal .

In [74]:
stats.levene(auto["harmonic_mileage"], manual["harmonic_mileage"])

LeveneResult(statistic=0.013901207169738269, pvalue=0.9062462281015493)

In [76]:
stats.ttest_ind(auto["average_mileage"], manual["average_mileage"], equal_var = False)

Ttest_indResult(statistic=-4.443514012903071, pvalue=1.795224899991793e-05)

The - test value and a p<0.05 suggests that manual cars have significantly better gas mileage on average. 

In [77]:
## with harmonic mileage
stats.ttest_ind(auto["harmonic_mileage"], manual["harmonic_mileage"], equal_var = False)


Ttest_indResult(statistic=-4.47444321386703, pvalue=1.598070270207952e-05)

# reveiw notes
I had to go back and re do the anlaysis using the harmonic mean, rather than arithmetic mean at first. 

Explanation on why I had to use a harmonic mean for gas mileage can be found here : https://machinelearningmastery.com/arithmetic-geometric-and-harmonic-means-for-machine-learning/